Recordar consultar el [Diccionario de datos](./Datasets/Google%20Maps/Diccionario%20de%20datos.docx)


# Importaciones


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import contextily as ctx
from shapely.geometry import Point
import geopy
from geopy.geocoders import Nominatim
from collections import Counter


# Llamado de datos a dataframes


In [ ]:
df_maps_restaurantes = pd.read_parquet(r"Generated\Google\metada_sitios.parquet")
df_maps_reviews = pd.read_parquet(r"Generated\Google\merge_site_reviews.parquet")
df_yelp_restaurantes = pd.read_parquet(r"Generated\Yelp\bussines.parquet")
df_yelp_reviews = pd.read_parquet(r"Generated\Yelp\review.parquet")

## Conteo de Registros

Elegimos la empresa Subway, por lo que contabilizaremos los registros presentes en los dataframes

In [ ]:
subway_yelp = df_yelp_restaurantes[df_yelp_restaurantes["name"] == "Subway"]
subway_maps = df_maps_restaurantes[df_maps_restaurantes["name"] == "Subway"]
total_subway_records = subway_yelp.shape[0] + subway_maps.shape[0]
print("Número de registros de Subway en restaurantes (Maps y Yelp):", total_subway_records)


Para revisar el dataframe de reviews debemos obtener el Business_id y usarlo de guía

Para eso, crearemos una lista que contenga todos los business_id que correspondan a locales de Subway, luego crearemos un dataframe que contenga todas las reviews de subway y uno que contenga todos los usuarios que realizaron reviews


In [ ]:
subway_business_ids = df_yelp_restaurantes.loc[df_yelp_restaurantes["name"] == "Subway", "business_id"].tolist()
reviews_yelp_subway = df_yelp_reviews[df_yelp_reviews["business_id"].isin(subway_business_ids)]

# Outliers


## df_maps_restaurantes

### num_of_reviews

Observamos la distribución de la columna `num_of_reviews`

In [ ]:
sns.boxplot(x=df_maps_restaurantes["num_of_reviews"], color="blue")
plt.xlabel("Valores")
plt.title("Distribución num_of_reviews")
plt.show()

Y decidimos eliminar datos que no sean representativos, siendo estos aquellos que posean mas de 160. Recortando asi un 3% `256` de los datos que poseiamos `9489`

In [ ]:
df_maps_restaurantes.drop(df_maps_restaurantes[df_maps_restaurantes["num_of_reviews"] > 160].index,inplace=True,)

### Latitude y Longitude

Graficamos los datos en un mapa de USA en busca de registros con coordenadas atipicas

In [ ]:
usa_geojson = "Geojson/us-states.json"
usa_gpd = gpd.read_file(usa_geojson)
registros_gdf = gpd.GeoDataFrame(df_maps_restaurantes,geometry=gpd.points_from_xy(df_maps_restaurantes["longitude"], df_maps_restaurantes["latitude"]),)

fig, ax = plt.subplots(figsize=(10, 8))
usa_gpd.plot(ax=ax, edgecolor="black", alpha=0.5)
registros_gdf.plot(ax=ax, color="red", markersize=1)
plt.title("Registros en el Mapa")
plt.xlabel("Longitud")
plt.ylabel("Latitud")
ax.set_xticks(range(-180, -60, 5))
ax.set_xticklabels(range(-180, -60, 5), rotation=45)
ax.grid(True)
plt.tight_layout()

plt.show()

In [ ]:
df_maps_restaurantes.loc[(df_maps_restaurantes["latitude"] < 21) & (df_maps_restaurantes["longitude"] < -70) & (df_maps_restaurantes["longitude"] > -80)]

Buscando en google la direccion devuelve las correctas coordenadas del local


In [ ]:
df_maps_restaurantes.loc[df_maps_restaurantes["gmap_id"] == "0x8ececf93aaa3553b:0x80d37fdf4ff72429","latitude",] = 43.127694
df_maps_restaurantes.loc[df_maps_restaurantes["gmap_id"] == "0x8ececf93aaa3553b:0x80d37fdf4ff72429","longitude",] = -89.363469

## df_yelp_restaurantes

Se realiza un boxplot de la puntiación de estrellas y de la cantidad de reviews para visualizar posibles outliers

Eliminacion de etiquetas de categoría obsoletos


In [ ]:
categorias = df_yelp_restaurantes["categories"].str.split(", ").explode()
frecuencia_categorias = Counter(categorias)
menos_repetidas = frecuencia_categorias.most_common()[:-168:-1]
print("Las menos repetidas son:")
for categoria, frecuencia in menos_repetidas[:20]:
    print(f"{categoria}: {frecuencia} veces")
print("La lista continúa...")


In [ ]:
categorias = df_yelp_restaurantes["categories"].str.split(", ").explode()
frecuencia_categorias = categorias.value_counts()
categorias_unicas = frecuencia_categorias[frecuencia_categorias == 1].index
df_yelp_restaurantes["categories"] = df_yelp_restaurantes["categories"].apply(lambda x: ", ".join(c for c in x.split(", ") if c not in categorias_unicas))

## Corrección de tipo de datos

In [ ]:
df_yelp_restaurantes.info()

In [ ]:
df_yelp_reviews.info()

Los formatos se encuentran en su mayoria en el estado correcto, se harán los siguientes cambios

- ### df_yelp_restaurantes:
  Se cambia el formato de la columna _is_open_ de Entero a Booleano puesto sus datos eran 1 y 0
- ### df_yelp_reviews:
  En las columnas _yelping_since_ (referencia a la fecha de creción del usuario) y _date_ (referencia de la fecha de posteo de la reseña) se les quita la hora puesto que es un dato obsoleto y se cambia su formato a Datetime


In [ ]:
df_yelp_restaurantes["is_open"] = df_yelp_restaurantes["is_open"].astype(bool)
df_yelp_reviews["date"] = pd.to_datetime(df_yelp_reviews["date"])

## Cruze de tablas


Filtramos las reviews dejando solo las que pertenecen a los locales que quedaron luego del tratamiento de outliers


In [ ]:
restaurantes = df_maps_restaurantes["gmap_id"].to_list()
df_maps_reviews = df_maps_reviews[df_maps_reviews["gmap_id"].isin(restaurantes)]

## df_maps_reviews


### num_of_reviews

Observamos la distribución de la columna `num_of_reviews`


In [ ]:
sns.boxplot(x=df_maps_reviews["num_of_reviews"], color="blue")
plt.xlabel("Valores")
plt.title("Distribución num_of_reviews")

plt.show()

Y decidimos eliminar datos que no sean representativos, siendo estos aquellos que posean mas de 160. Recortando asi un 3% `256` de los datos que poseiamos `9489`


In [ ]:
df_maps_restaurantes.drop(df_maps_restaurantes[df_maps_restaurantes["num_of_reviews"] > 160].index,inplace=True,)

### `latitude` y `longitude`


Graficamos los datos en un mapa de USA en busca de registros con coordenadas atipicas


In [ ]:
usa_geojson = "Geojson/us-states.json"
usa_gpd = gpd.read_file(usa_geojson)
registros_gdf = gpd.GeoDataFrame(
    df_maps_reviews,
    geometry=gpd.points_from_xy(
        df_maps_reviews["longitude"], df_maps_reviews["latitude"]
    ),
)

fig, ax = plt.subplots(figsize=(10, 8))
usa_gpd.plot(ax=ax, edgecolor="black", alpha=0.5)
registros_gdf.plot(ax=ax, color="red", markersize=1)
plt.title("Registros en el Mapa")
plt.xlabel("Longitud")
plt.ylabel("Latitud")
ax.set_xticks(range(-180, -60, 5))
ax.set_xticklabels(range(-180, -60, 5), rotation=45)
ax.grid(True)
plt.tight_layout()

plt.show()

# Tipos de datos


## df_maps_restaurantes


In [ ]:
df_maps_restaurantes.info()

Cambiamos el tipo de datos de las columnas `avg_rating`, `num_of_reviews` y `price` para reducir el uso de memoria a cambio de precisiòn innecesaria en los datos


In [ ]:
df_maps_restaurantes["avg_rating"] = df_maps_restaurantes["avg_rating"].astype("float32")
df_maps_restaurantes["num_of_reviews"] = df_maps_restaurantes["num_of_reviews"].astype("int32")
df_maps_restaurantes["price"] = df_maps_restaurantes["price"].astype("category")

## df_maps_reviews


Cambiamos el tipo de datos de las columnas para reducir el uso de memoria.


In [ ]:
df_maps_reviews_copy = df_maps_reviews.copy()
df_maps_reviews_copy["rating"] = df_maps_reviews_copy["rating"].astype("int8")
df_maps_reviews_copy["num_of_reviews"] = df_maps_reviews_copy["num_of_reviews"].astype(
    "int32"
)
df_maps_reviews_copy["price"] = df_maps_reviews_copy["price"].astype("category")
df_maps_reviews_copy["anio"] = df_maps_reviews_copy["anio"].astype("category")
df_maps_reviews_copy["estado"] = df_maps_reviews_copy["estado"].astype("category")
df_maps_reviews_copy["avg_rating"] = df_maps_reviews_copy["avg_rating"].astype(
    "float32"
)
df_maps_reviews_copy["state_ab"] = df_maps_reviews_copy["state_ab"].astype("category")
df_maps_reviews_copy["us_state"] = df_maps_reviews_copy["us_state"].astype("category")
df_maps_reviews = df_maps_reviews_copy

Para revisar el dataframe de reviews y el de usuarios, deberemos obtener el Business_id y usarlo de guía

Para eso, crearemos una lista que contenga todos los business_id que correspondan a locales de Subway, luego crearemos un dataframe que contenga todas las reviews de subway y uno que contenga todos los usuarios que realizaron reviews


In [ ]:
(df_yelp_restaurantes.loc[df_yelp_restaurantes["name"] == "Subway"]).head(1)

In [ ]:
subway_business_ids = df_yelp_restaurantes.loc[df_yelp_restaurantes["name"] == "Subway", "business_id"].tolist()
len(subway_business_ids)

In [ ]:
reviews_yelp_subway = df_yelp_reviews[df_yelp_reviews["business_id"].isin(subway_business_ids)]
reviews_yelp_subway

# Verificacion de valores nulos


Visualizamos los valores nulos de manera grafica con un mapa de calor


In [ ]:
sns.heatmap(df_maps_restaurantes.isnull(), cmap="viridis", cbar=False)
plt.title("Mapa de calor de valores faltantes")
plt.show()

Analizamos el porcentaje de valores nulos presentes en cada columna


In [ ]:
porcnull_rest = df_maps_restaurantes.isnull().mean() * 100
columnasnull_rest = porcnull_rest[porcnull_rest > 0]
columnasnull_rest

Desechamos las columnas irrelevantes con un porcentaje alto de valores nulos


In [ ]:
df_maps_rest_mod = df_maps_restaurantes.drop(["description", "price"], axis=1)

Se rellenan los valores nulos


In [ ]:
df_maps_rest_mod["hours"] = df_maps_rest_mod["hours"].fillna("").apply(list)
df_maps_rest_mod["state"] = df_maps_rest_mod["state"].fillna("N/A")
df_maps_rest_mod["address"] = df_maps_rest_mod["address"].fillna("N/A")
df_maps_rest_mod["MISC"] = df_maps_rest_mod["MISC"].fillna("").apply(dict)
df_maps_rest_mod["relative_results"] = df_maps_rest_mod["relative_results"].fillna("").apply(list)


Se verifica la ausencia de valores nulos


In [ ]:
sns.heatmap(df_maps_rest_mod.isnull(), cmap="viridis", cbar=False)
plt.title("Mapa de calor de valores faltantes")
plt.show()

Se realiza el mismo procedimiento


In [ ]:
sns.heatmap(df_maps_reviews.isnull(), cmap="viridis", cbar=False)
plt.title("Mapa de calor de valores faltantes")
plt.show()

In [ ]:
porcnull_review = df_maps_reviews.isnull().mean() * 100
columnasnull_review = porcnull_review[porcnull_review > 0]
columnasnull_review

In [ ]:
df_maps_reviews_mod = df_maps_reviews.drop(["pics", "resp"], axis=1)

In [ ]:
df_maps_reviews_mod["text"] = df_maps_reviews_mod["text"].fillna("N/A")
df_maps_reviews_mod["address"] = df_maps_reviews_mod["address"].fillna("N/A")
df_maps_reviews_mod["description"] = df_maps_reviews_mod["description"].fillna("N/A")
df_maps_reviews_mod["hours"] = df_maps_reviews_mod["hours"].fillna("").apply(list)
df_maps_reviews_mod["MISC"] = df_maps_reviews_mod["MISC"].fillna("").apply(dict)
df_maps_reviews_mod["state"] = df_maps_reviews_mod["state"].fillna("N/A")
df_maps_reviews_mod["relative_results"] = df_maps_reviews_mod["relative_results"].fillna("").apply(list)


In [ ]:
sns.heatmap(df_maps_reviews_mod.isnull(), cmap="viridis", cbar=False)
plt.title("Mapa de calor de valores faltantes")
plt.show()